In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# load
df=pd.read_csv('quebec.dat', sep='\t')

# create uid column - 'person_id'
df['person_id'] = df.index

In [89]:
len(df)

3090

In [90]:
# wide to long
df_long = pd.wide_to_long(df, stubnames=["op_cost", "fix_cost", "cost_inc", "avail"], i="person_id", j="alt")

# print(len(df_long))
# print(df_long.head())

# reset index for clarity
df_long = df_long.reset_index()

# get y
df_long["chosen"] = (df_long["alt"] == df_long["choice"]).astype(int)

# Retain only available alternatives
df_long = df_long[df_long["avail"] == 1]

# print(len(df_long))


In [91]:
#df_long.tail(20)

In [92]:
import statsmodels.api as sm
# create alternative-specific constants (ASCs)
df_long = pd.get_dummies(df_long, columns=["alt"], drop_first=True)


In [93]:
# design matrix w/: op_cost, fix_cost, cost_inc + const
X_restricted = df_long[["op_cost", "fix_cost", "cost_inc"]]
X_restricted = sm.add_constant(X_restricted)  # Add intercept

# fit
model_restricted = sm.MNLogit(y, X_restricted)
result_restricted = model_restricted.fit()

result_restricted.summary()


Optimization terminated successfully.
         Current function value: 0.249702
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                 chosen   No. Observations:                22798
Model:                        MNLogit   Df Residuals:                    22794
Method:                           MLE   Df Model:                            3
Date:                Thu, 20 Mar 2025   Pseudo R-squ.:                  0.3707
Time:                        13:03:59   Log-Likelihood:                -5692.7
converged:                       True   LL-Null:                       -9045.8
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
  chosen=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4913      0.089     16.825      0.000       1.318       1.665
op_cost        0.4749      0.056      8.530      0.000       0.366       0.584
fix_cost      -6.6930      0.133    -50.463      0.000      -6.953      -6.433
cost_inc       0.0704      0.020      3.544      0.000       0.031       0.109
==============================================================================
"""

In [94]:
X_unrestricted = df_long[["op_cost", "fix_cost", "cost_inc", "nb_rooms", "surface", "age", "income"]]
X_unrestricted = sm.add_constant(X_unrestricted)

# fit
model_unrestricted = sm.MNLogit(y, X_unrestricted)
result_unrestricted = model_unrestricted.fit()

result_unrestricted.summary()

Optimization terminated successfully.
         Current function value: 0.249358
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                 chosen   No. Observations:                22798
Model:                        MNLogit   Df Residuals:                    22790
Method:                           MLE   Df Model:                            7
Date:                Thu, 20 Mar 2025   Pseudo R-squ.:                  0.3715
Time:                        13:04:00   Log-Likelihood:                -5684.9
converged:                       True   LL-Null:                       -9045.8
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
  chosen=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.7637      0.168     10.484      0.000       1.434       2.093
op_cost        0.4135      0.070      5.868      0.000       0.275       0.552
fix_cost      -7.2911      0.222    -32.794      0.000      -7.727      -6.855
cost_inc       0.2094      0.045      4.613      0.000       0.120       0.298
nb_rooms       0.0132      0.017      0.783      0.433      -0.020       0.046
surface        0.0517      0.038      1.351      0.177      -0.023       0.127
age            0.0133      0.019      0.713      0.476      -0.023       0.050
income        -0.0975      0.028     -3.512      0.000      -0.152      -0.043
==============================================================================
"""

In [95]:
import scipy.stats as stats

# get llf
LL_restricted = result_restricted.llf
LL_unrestricted = result_unrestricted.llf

LR_stat = -2 * (LL_restricted - LL_unrestricted)

# get the degrees of freedom
df_restricted = X_restricted.shape[1] 
df_unrestricted = X_unrestricted.shape[1]
df_diff = df_unrestricted - df_restricted  # diff in deg of freedom

# get p value
p_value = 1 - stats.chi2.cdf(LR_stat, df_diff)

# Display results
LR_stat, df_diff, p_value


(15.672367976550959, 4, 0.0034918315522377563)

With p<0.05 as the given significance level, the null hypothesis should be rejected,\
concluding that the demographic variables significantly improves the model fit.